In [67]:
import imaplib
import base64
import os
import email
import xlrd
import xlwt
import openpyxl
from email.message import EmailMessage

In [68]:
class Email:
    def __init__(self, sender, subject, body):
        self.sender = sender
        self.subject = subject
        self.body = body
        
    def summary(self):
        print("Email From: " + self.sender)
        print("Email Subject: " + self.subject)
        print("Email Body: " + self.body)


In [69]:
#Set up for connection to outlook IMAP server
imap_clients = {
    "outlook" : ("imap-mail.outlook.com", 993)
}
loginFile = open("credentials.txt", "r")
connectionInformation = {}
for line in loginFile:
    key = line.split(":")[0]
    value = line.split(":")[1].rstrip("\n")
    connectionInformation[key] = value

client = connectionInformation["client"]
#Connect and login to outlook mailbox
mailbox = imaplib.IMAP4_SSL(imap_clients[client][0], imap_clients[client][1])
username = connectionInformation["username"]
password = connectionInformation["password"]
mailbox.login(username, password)

('OK', [b'LOGIN completed.'])

In [70]:
#Set the current mailbox to search from
mailbox.select('Inbox')
#Start going through unread emails 
unread_emails = []
type, data = mailbox.search(None, '(UNSEEN)')
mail_ids = data[0]
id_list = mail_ids.split()
temp = data[0].split()
if type == "OK":
    for num in temp:
        typ, data = mailbox.fetch(num, '(RFC822)' )
        raw_email = data[0][1]
        for response_part in data:
            if isinstance(response_part, tuple):
                #Create new Email object and add it to unread_emails array
                msg = email.message_from_string(response_part[1].decode('utf-8'))
                email_subject = msg['subject']
                email_from = msg['from']
                email_body = msg.get_payload()[0].get_payload()
                unread_email = EmailMessage()
                unread_email['Subject'] = email_subject
                unread_email['From'] = email_from
                unread_email.set_content(email_body)
                unread_emails.append(unread_email)

In [71]:
#for email in unread_emails:
#    print(email.summary())
#body = msg.get_payload()[0].get_payload()
#data_index = body.index("Name")
#body = body[data_index:].rsplit("\r\n")
#body

In [72]:
mailbox.close()
mailbox.logout()

('BYE', [b'Microsoft Exchange Server IMAP4 server signing off.'])

In [73]:
workbook = openpyxl.load_workbook("templates.xlsx")
customers = workbook["Customers"]

In [74]:
for email in unread_emails:
    email_body = email.get_content()
    email_body = email_body.replace("\n\n", " ")
    email_body = email_body.replace("\n", " ")
    email_body = email_body.replace(": ", ":")
    param_dict = {}
    key = ""
    value = ""
    onValue = False
    onKey = True
    onComments = False
    for character in email_body:
        if onKey:
            if character == ':':
                if key == "Comments":
                    onComments = True
                onValue = True
                onKey = False
                continue
            else:
                key = key + character
        elif onValue:
            if not onComments:
                if character == ' ':
                    onValue = False
                    onKey = True
                    param_dict[key] = value
                    key = ""
                    value = ""
                else:
                    value = value + character
            elif email_body.endswith(character):
                value = value + " "
            else:
                value = value + character
    param_dict[key] = value            
    print(param_dict)
    #Insert information into excel sheet 
    key_dict = []
    for row in customers.iter_cols(min_row=1, max_col=customers.max_column, max_row=2, values_only=True):
        key_dict.append(row[0])

    new_customer_row = customers.max_row + 1
    for current_col in range(customers.max_column):
        if key_dict[current_col] in param_dict:
            customers.cell(row=new_customer_row, column=current_col + 1).value = param_dict[key_dict[current_col]]

{'First Name': 'Antony', 'Last Name': 'Darsuma', 'Source': 'codeninjas.com', 'Primary Email': 'icerain@uwalumni.com', 'Primary Phone': '3464123526', 'City, State': ',', 'Child First Name': 'Julius', 'Child Last Name': 'Darsuma', 'Comments': 'Additional Information Child:Julius Darsuma Referrer:https://www.codeninjas.com/tx-cypress '}
{'First Name': 'Jes', 'Last Name': 'Byboth', 'Source': 'codeninjas.com', 'Primary Email': 'jesbyboth@gmail.com', 'Primary Phone': '9794504780', 'City, State': ',', 'Child First Name': 'Aiden', 'Child Last Name': 'Byboth', 'Comments': 'Additional Information Inquired About:Code Ninjas CREATE Program Slug:code-ninjas-create Child:Aiden Byboth Referrer:https://www.codeninjas.com/tx-cypress/code-ninjas-create/request-information '}


In [75]:
workbook.save("templates.xlsx")